SBG - Spectral Resample Process - Application Notebook

In [1]:
import json
import shutil
import sys
import os
import hytools as ht
from hytools.io.envi import WriteENVI
import numpy as np
from scipy.interpolate import interp1d
from skimage.util import view_as_blocks
from PIL import Image
from pathlib import Path
import glob
import spectral.io.envi as envi
import datetime as dt
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

# stage_in packages
from unity_sds_client.resources.collection import Collection

/opt/conda/envs/sister/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-15 21:35:37,553	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Resampling Code

In [2]:
def generate_metadata(in_file,out_file,metadata):

    with open(in_file, 'r') as in_obj:
        in_met =json.load(in_obj)

    for key,value in metadata.items():
        in_met[key] = value

    with open(out_file, 'w') as out_obj:
        json.dump(in_met,out_obj,indent=3)

def gaussian(x,mu,fwhm):

    c = fwhm/(2* np.sqrt(2*np.log(2)))
    return np.exp(-1*((x-mu)**2/(2*c**2)))

def resample(in_file,out_file):

    image = ht.HyTools()
    image.read_file(in_file,'envi')

    if image.wavelengths.max()< 1100:
        new_waves = np.arange(400,991,10)
    else:
        new_waves = np.arange(400,2501,10)

    bins = int(np.round(10/np.diff(image.wavelengths).mean()))
    agg_waves  = np.nanmean(view_as_blocks(image.wavelengths[:(image.bands//bins) * bins],
                                           (bins,)),axis=1)

    if bins ==1 :
        agg_fwhm  = image.fwhm

    else:
        hi_res_waves = np.arange(300,2600)
        fwhm_array = np.zeros((image.bands,2600-300))

        for i,(wave,fwhm) in enumerate(zip(image.wavelengths,image.fwhm)):
            fwhm_array[i] = gaussian(hi_res_waves,wave,fwhm)

        sum_fwhm  = np.nansum(view_as_blocks(fwhm_array[:(image.bands//bins) * bins],
                                               (bins,2600-300)),axis=(1,2))
        agg_fwhm = []
        for i in range(len(agg_waves)):
            arg_max = np.argmax(sum_fwhm[i])
            half_max =  sum_fwhm[i].max()/2
            diff = np.abs(sum_fwhm[i]-half_max)
            end = arg_max + np.argmin(diff[arg_max:])
            start = np.argmin(diff[:arg_max])
            agg_fwhm.append(hi_res_waves[end] - hi_res_waves[start])

    #True resampled FWHM is difficult to determine, using a simple nearest neighbor approximation
    resampled_fwhm = interp1d(agg_waves,agg_fwhm,fill_value = 'extrapolate', kind = 'nearest')(new_waves)

    print(f"Aggregating every {bins} bands")

    out_header = image.get_header()
    out_header['bands'] = len(new_waves)
    out_header['wavelength'] = new_waves.tolist()
    out_header['fwhm'] = resampled_fwhm
    out_header['default bands'] = []

    if  "UNC" in in_file:
        out_header['description'] ='10 nm resampled reflectance uncertainty'
    else:
        out_header['description'] ='10 nm resampled reflectance'

    writer = WriteENVI(out_file,out_header)
    iterator =image.iterate(by = 'line')

    while not iterator.complete:
        line = iterator.read_next()[:,:(image.bands//bins) * bins]
        line  = np.nanmean(view_as_blocks(line,(1,bins,)),axis=(2,3))
        interpolator = interp1d(agg_waves,line,fill_value = 'extrapolate', kind = 'cubic')
        line = interpolator(new_waves)
        writer.write_line(line,iterator.current_line)

def generate_quicklook(input_file):

    img = ht.HyTools()
    img.read_file(input_file)
    image_file = input_file.replace('.bin','.png')

    if 'DESIS' in img.base_name:
        band3 = img.get_wave(560)
        band2 = img.get_wave(850)
        band1 = img.get_wave(660)
    else:
        band3 = img.get_wave(560)
        band2 = img.get_wave(850)
        band1 = img.get_wave(1660)

    rgb=  np.stack([band1,band2,band3])
    rgb[rgb == img.no_data] = np.nan

    rgb = np.moveaxis(rgb,0,-1).astype(float)
    bottom = np.nanpercentile(rgb,5,axis = (0,1))
    top = np.nanpercentile(rgb,95,axis = (0,1))
    rgb = np.clip(rgb,bottom,top)
    rgb = (rgb-np.nanmin(rgb,axis=(0,1)))/(np.nanmax(rgb,axis= (0,1))-np.nanmin(rgb,axis= (0,1)))
    rgb = (rgb*255).astype(np.uint8)

    im = Image.fromarray(rgb)
    im.save(image_file)

Inputs and Configurations

In the original pre-process, inputs are supplied by a run_config file. This consists of 2 entries (a reflectance file, uncertainty file, and a CRID).

In the Unity system, the data files required will be staged in for the application, and the crid is a config item that is passed in. To make this work in Unity, we will also pass in an "output collection" which is needed if we want to "persist" the output products in the data catalog.


In [4]:
#Retrieve Reflectance Dataset
input_resample_dataset    = '/unity/ads/input_collections/SBG-L2A-RESAMPLE/catalog.json' # type: stage-in
output_stac_catalog_dir   = '/unity/ads/outputs/SBG-L2A-RESAMPLE/' # type: stage-out

output_collection_name    = 'example-L2A-Resample-Collect'

#Pre-process variables
#From the config.json, retrieve the following information:
crid = "000" #hardcoded but will be passed in 

Import Files from STAC Item Collection

In [5]:
inp_collection = Collection.from_stac(input_resample_dataset)
data_filenames = inp_collection.data_locations()

data_filenames

['/unity/ads/input_collections/SBG-L2A-RESAMPLE/SISTER_EMIT_L2A_RFL_20231206T160939_001.bin',
 '/unity/ads/input_collections/SBG-L2A-RESAMPLE/SISTER_EMIT_L2A_RFL_20231206T160939_001_UNC.bin']

Get the data files from the STAC files

In [6]:
for f in data_filenames:
    if "_UNC.bin" in f:
        unc_base_name = Path(f).stem
        unc_file = f
    elif ".bin" in f:
        rfl_base_name = Path(f).stem
        rfl_file = f

print(rfl_base_name)
print(unc_base_name)

sister,sensor,level,product,datetime,in_crid = rfl_base_name.split('_')

rfl_met = rfl_file.replace('.bin','.met.json')

out_rfl_file =  f'{output_stac_catalog_dir}/SISTER_{sensor}_L2A_RSRFL_{datetime}_{crid}.bin'
out_rfl_met = out_rfl_file.replace('.bin','.met.json')

print("Reflectance File:" + rfl_file)
print("Output path:" + out_rfl_met)


sister,sensor,level,product,datetime,in_crid,subproduct = unc_base_name.split('_')

unc_met = unc_file.replace('.bin','.met.json')

out_unc_file =  f'{output_stac_catalog_dir}/SISTER_{sensor}_L2A_RSRFL_{datetime}_{crid}_UNC.bin'
out_unc_met = out_unc_file.replace('.bin','.met.json')

print("Uncertainty File:" + unc_base_name)
print("Output path:" + out_unc_met)


Reflectance File:/unity/ads/input_collections/SBG-L2A-RESAMPLE/SISTER_EMIT_L2A_RFL_20231206T160939_001.bin
Output path:/unity/ads/outputs/SBG-L2A-RESAMPLE//SISTER_EMIT_L2A_RSRFL_20231206T160939_000.met.json
Uncertainty File:SISTER_EMIT_L2A_RFL_20231206T160939_001_UNC
Output path:/unity/ads/outputs/SBG-L2A-RESAMPLE//SISTER_EMIT_L2A_RSRFL_20231206T160939_000_UNC.met.json


Resampling Commands

In [7]:
print("Executing resampling alogrithm on Reflectance file...")
resample(rfl_file, out_rfl_file)

print("Executing resampling alogrithm on Uncertainty file...")
resample(unc_file, out_unc_file)

print("Generating quicklook on Reflectance file...")
generate_quicklook(out_rfl_file)

Executing resampling alogrithm on Reflectance file...
Aggregating every 1 bands
Executing resampling alogrithm on Uncertainty file...
Aggregating every 1 bands
Generating quicklook on Reflectance file...


Create stage-out item catalog

In [20]:
#Uncertain about this part. What is supposed to get sent out to STAC? 
from datetime import datetime, timezone

# Create a collection
out_collection = Collection(output_collection_name)

# Add output file(s) to the dataset
file = glob.glob(f"{output_stac_catalog_dir}/*{crid}*.hdr")

if file:
    header = envi.read_envi_header(file[0])    
    start_time = dt.datetime.strptime(header['start acquisition time'], "%Y-%m-%dt%H:%M:%Sz")
    end_time = dt.datetime.strptime(header['end acquisition time'], "%Y-%m-%dt%H:%M:%Sz")
    # Create a Dataset for the collection
    name = os.path.splitext(os.path.basename(file[0]))[0]
    dataset = Dataset(
        name=name,
        collection_id=out_collection.collection_id, 
        start_time=start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        end_time=end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        creation_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
        )
    
    for file in glob.glob(f"{output_stac_catalog_dir}/*{crid}*"):  

        if file.endswith(".bin"):
            dataset.add_data_file(DataFile("binary", file, ["data"]))
        elif file.endswith(".png"):
            dataset.add_data_file(DataFile("image/png", file, ["browse"]))
        elif file.endswith(".hdr"):
            dataset.add_data_file(DataFile("header", file, ["data"]))
        else:
            dataset.add_data_file(DataFile(None, file, ["metadata"]))

    dataset.add_data_file(DataFile("text/json", output_stac_catalog_dir + '/' +  name +'.json', ["metadata"]))


# Add the dataset to the collection
out_collection._datasets.append(dataset)



In [21]:
Collection.to_stac(out_collection, output_stac_catalog_dir)